In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
log_df = pd.read_csv('new_log.csv')

In [3]:
log_df = log_df.sort_values(by='msg_lower')

In [4]:
from drain3 import TemplateMiner #开源在线日志解析框架
from drain3.file_persistence import FilePersistence
from drain3.template_miner_config import TemplateMinerConfig

In [5]:
config = TemplateMinerConfig()
config.load('./drain3.ini') ## 这个文件在drain3的github仓库里有
config.profiling_enabled = False

In [6]:
drain_file = 'comp_a_sellog'
persistence = FilePersistence(drain_file + '.bin')
template_miner = TemplateMiner(persistence, config=config)

In [7]:
for msg in log_df.msg_lower.tolist():
    template_miner.add_log_message(msg)
temp_count = len(template_miner.drain.clusters)

In [8]:
template_dic = {}

for cluster in template_miner.drain.clusters:
    template_dic[cluster.cluster_id] = cluster.size

In [9]:
temp_count_f = len(template_dic)

In [10]:
temp_count_f

187

In [11]:
def match_template(df, template_miner, template_dic):
    msg = df.msg_lower
    cluster = template_miner.match(msg) # 匹配模板，由开源工具提供
    if cluster and cluster.cluster_id in template_dic:
        df['template_id'] = cluster.cluster_id # 模板id
        df['template'] = cluster.get_template() # 具体模板
    else:
        df['template_id'] = 'None' # 没有匹配到模板的数据也会记录下来，之后也会用作一种特征。
        df['template'] = 'None'
    return df

In [12]:
log_df = log_df.apply(match_template, template_miner=template_miner, template_dic=template_dic, axis=1)

In [13]:
log_df.to_csv('log_template.csv', index=False)